This notebook collects all the damages in the baseline and adapted conditions and calculates the expected annual damages: direct, indirect tributary, and indirect total for the network. It also compiles the adaptation costs and calculates the benefit cost ratio

In [11]:
import pickle
import os
import sys
import numpy as np
import config
from pathlib import Path
import pathlib
from config import configparser
import pandas as pd

In [2]:
# Load configuration with ini file (created running config.py)
config_file=r'C:\repos\ci_adapt\config_ci_adapt.ini'
config = configparser.ConfigParser()
config.read(config_file)

p = Path('..')
hazard_type = config.get('DEFAULT', 'hazard_type')
infra_type = config.get('DEFAULT', 'infra_type')
country_code = config.get('DEFAULT', 'country_code')
country_name = config.get('DEFAULT', 'country_name')
hazard_data_subfolders = config.get('DEFAULT', 'hazard_data_subfolders')
asset_data = config.get('DEFAULT', 'asset_data')
vulnerability_data = config.get('DEFAULT', 'vulnerability_data')
data_path = Path(pathlib.Path.home().parts[0]) / 'Data'
interim_data_path = data_path / 'interim' / 'collected_flood_runs'

In [12]:
#load pickled shortest paths, disrupted edges, shortest paths, graph
shortest_paths = pickle.load(open(data_path / 'interim' / 'indirect_damages' / 'shortest_paths.pkl', 'rb'))
disrupted_edges_by_basin = pickle.load(open(data_path / 'interim' / 'indirect_damages' / 'disrupted_edges_by_basin.pkl', 'rb'))
graph_r0 = pickle.load(open(data_path / 'interim' / 'indirect_damages' / 'graph_0.pkl', 'rb'))
disrupted_shortest_paths = pickle.load(open(data_path / 'interim' / 'indirect_damages' / 'disrupted_shortest_paths.pkl', 'rb'))
event_impacts = pickle.load(open(data_path / 'interim' / 'indirect_damages' / 'event_impacts.pkl', 'rb'))
print('Loaded data from baseline impact assessment')

if 'collect_output' not in locals():
    collect_output_path = f'C:/Data/interim/collected_flood_runs/sample_collected_run.pkl'
    with open(collect_output_path, 'rb') as f:
        collect_output = pickle.load(f)

# Load results
direct_damages_adapted_path = data_path / 'output' / 'adapted_direct_damages.pkl'
indirect_damages_adapted_path = data_path / 'output' / 'adapted_indirect_damages.pkl'
adaptations_df_path = data_path / 'output' / 'adaptations.csv'


with open(direct_damages_adapted_path, 'rb') as f:
    direct_damages_adapted = pickle.load(f)
with open(indirect_damages_adapted_path, 'rb') as f:
    indirect_damages_adapted = pickle.load(f)
adaptations_df = pd.read_csv(adaptations_df_path)



Loaded data from baseline impact assessment


In [13]:
direct_damages_baseline_sum = {key: (sum(v[0] for v in collect_output[key].values()), sum(v[1] for v in collect_output[key].values())) for key in collect_output if key in direct_damages_adapted}
direct_damages_adapted_sum = {key: (sum(v[0] for v in direct_damages_adapted[key].values()), sum(v[1] for v in direct_damages_adapted[key].values())) for key in direct_damages_adapted}
direct_damages_diff = {key: (direct_damages_baseline_sum[key][0] - direct_damages_adapted_sum[key][0], direct_damages_baseline_sum[key][1] - direct_damages_adapted_sum[key][1]) for key in direct_damages_baseline_sum}
direct_damages_diff

{'flood_DERP_RW_H_4326_2080410170': (0.0, 0.0),
 'flood_DERP_RW_H_4326_2080410430': (0.0, 0.0),
 'flood_DERP_RW_H_4326_2080410540': (0.0, 0.0),
 'flood_DERP_RW_H_4326_2080410660': (0, 0),
 'flood_DERP_RW_H_4326_2080410760': (0, 0),
 'flood_DERP_RW_H_4326_2080411370': (0.0, 0.0),
 'flood_DERP_RW_H_4326_2080416200': (0, 0),
 'flood_DERP_RW_H_4326_2080416210': (0.0, 0.0),
 'flood_DERP_RW_H_4326_2080418600': (0.0, 0.0),
 'flood_DERP_RW_H_4326_2080418720': (0, 0),
 'flood_DERP_RW_H_4326_2080418880': (0.0, 0.0),
 'flood_DERP_RW_H_4326_2080418890': (0.0, 0.0),
 'flood_DERP_RW_H_4326_2080419070': (0, 0),
 'flood_DERP_RW_H_4326_2080419180': (0.0, 0.0),
 'flood_DERP_RW_H_4326_2080420240': (0.0, 0.0),
 'flood_DERP_RW_H_4326_2080420340': (0, 0),
 'flood_DERP_RW_H_4326_2080421680': (0.0, 0.0),
 'flood_DERP_RW_H_4326_2080421770': (0.0, 0.0),
 'flood_DERP_RW_H_4326_2080428080': (0, 0),
 'flood_DERP_RW_H_4326_2080428160': (0.0, 0.0),
 'flood_DERP_RW_H_4326_2080428500': (0, 0),
 'flood_DERP_RW_H_4326_2

In [14]:
event_impacts = {key: event_impacts[key] if key in event_impacts.keys() else 0.0 for key in collect_output.keys()}
event_impacts_diff = {key: event_impacts[key] - indirect_damages_adapted[key] for key in event_impacts if key in indirect_damages_adapted}

event_impacts_diff


{'flood_DERP_RW_H_4326_2080410170': 0.0,
 'flood_DERP_RW_H_4326_2080410430': 0.0,
 'flood_DERP_RW_H_4326_2080410540': 0.0,
 'flood_DERP_RW_H_4326_2080410660': 0.0,
 'flood_DERP_RW_H_4326_2080410760': 0.0,
 'flood_DERP_RW_H_4326_2080411370': 0.0,
 'flood_DERP_RW_H_4326_2080416200': 0.0,
 'flood_DERP_RW_H_4326_2080416210': 0.0,
 'flood_DERP_RW_H_4326_2080418600': 0.0,
 'flood_DERP_RW_H_4326_2080418720': 0.0,
 'flood_DERP_RW_H_4326_2080418880': 0.0,
 'flood_DERP_RW_H_4326_2080418890': 0.0,
 'flood_DERP_RW_H_4326_2080419070': 0.0,
 'flood_DERP_RW_H_4326_2080419180': 0.0,
 'flood_DERP_RW_H_4326_2080420240': 0.0,
 'flood_DERP_RW_H_4326_2080420340': 0.0,
 'flood_DERP_RW_H_4326_2080421680': 0.0,
 'flood_DERP_RW_H_4326_2080421770': 0.0,
 'flood_DERP_RW_H_4326_2080428080': 0.0,
 'flood_DERP_RW_H_4326_2080428160': 0.0,
 'flood_DERP_RW_H_4326_2080428500': 0.0,
 'flood_DERP_RW_H_4326_2080429430': 0.0,
 'flood_DERP_RW_H_4326_2080429540': 0.0,
 'flood_DERP_RW_H_4326_2080429670': 0.0,
 'flood_DERP_RW_

In [15]:
# direct_damages_adapted
def compare_outputs(collect_output, direct_damages_adapted, event_impacts, indirect_damages_adapted):
    changes = 0
    for key in collect_output.keys():
        print('direct: ', key)
        print(collect_output[key])
        print(direct_damages_adapted[key])
        no_change_direct = collect_output[key] == direct_damages_adapted[key]
        if no_change_direct == False:
            changes += 1
        print('No change direct: ', no_change_direct, end='\n')
        print('indirect: ')
        print(event_impacts[key] if key in event_impacts else None)
        print(indirect_damages_adapted[key] if key in indirect_damages_adapted else None)
        if key in event_impacts and key in indirect_damages_adapted:
            no_change_indirect = event_impacts[key] == indirect_damages_adapted[key]
            if no_change_indirect == False:
                changes += 1
            print('No change: ', no_change_indirect, end='\n\n')
        else:
            print('No change: ', False, end='\n\n')
    return print('changes: ', changes)

compare_outputs(collect_output, direct_damages_adapted, event_impacts, indirect_damages_adapted)

direct:  flood_DERP_RW_H_4326_2080410170
{7717: (0, 0), 7735: (0, 0), 7901: (840.3763964725075, 840.3763964725075), 20510: (0, 0), 46830: (0, 0), 46833: (0, 0), 46835: (0, 0), 46836: (0, 0), 46837: (0, 0), 46839: (0, 0), 46851: (0, 0), 46852: (0, 0), 46856: (0, 0), 46857: (0, 0), 46862: (0, 0), 46868: (0, 0), 46869: (0, 0), 46871: (0, 0), 46875: (4209.840874290893, 9170.290556478123), 46876: (0, 0), 46877: (0, 0), 68516: (0, 0), 68517: (0, 0), 68521: (0, 0), 69993: (0, 0), 70116: (0, 0), 70118: (3610.1290746663167, 3610.1290746663167), 86898: (0, 0), 111875: (0, 0), 111876: (0, 0)}
{7717: (0, 0), 7735: (0, 0), 7901: (840.3763964725075, 840.3763964725075), 20510: (0, 0), 46830: (0, 0), 46833: (0, 0), 46835: (0, 0), 46836: (0, 0), 46837: (0, 0), 46839: (0, 0), 46851: (0, 0), 46852: (0, 0), 46856: (0, 0), 46857: (0, 0), 46862: (0, 0), 46868: (0, 0), 46869: (0, 0), 46871: (0, 0), 46875: (4209.840874290893, 9170.290556478123), 46876: (0, 0), 46877: (0, 0), 68516: (0, 0), 68517: (0, 0), 6852

In [ ]:
# Define other functions (development)
def find_basin_lists(basins, regions):
    intersect_basins_regions = gpd.overlay(basins, regions, how='intersection')
    exclude_main_rivers=intersect_basins_regions.loc[intersect_basins_regions['ORDER']==1]

    basins_exclusion_list = [x for x in exclude_main_rivers['HYBAS_ID'].values]
    basin_list_tributaries = set([x for x in intersect_basins_regions['HYBAS_ID'].values if x not in basins_exclusion_list]) 
    basin_list_full_flood = set(intersect_basins_regions['HYBAS_ID'].values)
    return basin_list_tributaries, basin_list_full_flood



basins_path = gpd.read_file(data_path / r'Floods\basins\hybas_eu_lev01-12_v1c\hybas_eu_lev08_v1c_valid.shp')
regions_path = gpd.read_file(data_path / r'QGIS_data\rhineland_palatinate.geojson')

basin_list_tributaries, basin_list_full_flood = find_basin_lists(basins_path, regions_path)

In [21]:
pd.options.display.float_format = "{:,.2f}".format

total_damages_adapted={}
for hazard_map in direct_damages_adapted.keys():

    map_rp_spec = hazard_map.split('_')[-3]

    adap_costs=adaptations_df['adaptation_cost']
    summed_adaptation_costs = sum(adap_costs)

    #direct damages
    summed_dd_bl_lower=direct_damages_baseline_sum[hazard_map][0]
    summed_dd_bl_upper=direct_damages_baseline_sum[hazard_map][1]

    summed_dd_ad_lower=direct_damages_adapted_sum[hazard_map][0]
    summed_dd_ad_upper=direct_damages_adapted_sum[hazard_map][1]
    
    #indirect damages
    if hazard_map not in event_impacts.keys():
        print(f'{hazard_map} not in event_impacts')
        id_bl=0
        id_ad=0
        id_ad_cleaned=0
    else:
        id_bl=event_impacts[hazard_map]
        id_ad=indirect_damages_adapted[hazard_map]
        id_ad_cleaned = 0 if id_ad == 99999999999999 else id_ad

    total_damages_adapted[hazard_map]=(map_rp_spec, summed_adaptation_costs, (summed_dd_bl_lower, summed_dd_bl_upper), (summed_dd_ad_lower, summed_dd_ad_upper), id_bl, id_ad_cleaned)
    
total_damages_adapted_df=pd.DataFrame(total_damages_adapted)
total_damages_adapted_df=total_damages_adapted_df.T
total_damages_adapted_df.columns=['return_period','summed_adaptation_costs', 'summed_dd_bl', 'summed_dd_ad', 'indirect damage baseline [€]', 'indirect damage adapted [€]']

# round and turn to million euros for reporting
total_damages_adapted_df_mill=total_damages_adapted_df.copy()
total_damages_adapted_df_mill['summed_adaptation_costs [M€]']=total_damages_adapted_df_mill['summed_adaptation_costs']/1e6
total_damages_adapted_df_mill['summed_dd_bl [M€]']=total_damages_adapted_df_mill['summed_dd_bl'].apply(lambda x: (x[0]/1e6, x[1]/1e6))
total_damages_adapted_df_mill['summed_dd_ad [M€]']=total_damages_adapted_df_mill['summed_dd_ad'].apply(lambda x: (x[0]/1e6, x[1]/1e6))
total_damages_adapted_df_mill['indirect damage baseline [M€]']=total_damages_adapted_df_mill['indirect damage baseline [€]']/1e6
total_damages_adapted_df_mill['indirect damage adapted [M€]']=total_damages_adapted_df_mill['indirect damage adapted [€]']/1e6
total_damages_adapted_df_mill.drop(['summed_adaptation_costs','summed_dd_bl', 'summed_dd_ad', 'indirect damage baseline [€]', 'indirect damage adapted [€]'], axis=1, inplace=True)



custom_order = ['H', 'M', 'L', 'Unknown']

total_damages_adapted_df_mill['return_period'] = pd.Categorical(total_damages_adapted_df_mill['return_period'], 
                                                                categories=custom_order, ordered=True)
sorted_total_damages_adapted_df_mill = total_damages_adapted_df_mill.sort_values(by='return_period', ascending=True)
sorted_total_damages_adapted_df_mill

,return_period,summed_adaptation_costs [M€],summed_dd_bl [M€],summed_dd_ad [M€],indirect damage baseline [M€],indirect damage adapted [M€]
flood_DERP_RW_H_4326_2080410170,H,0.00,"(0.008660346345429717, 0.013620796027616946)","(0.008660346345429717, 0.013620796027616946)",0.47,0.47
flood_DERP_RW_H_4326_2080433740,H,0.00,"(0.0, 0.0)","(0.0, 0.0)",0.00,0.00
flood_DERP_RW_H_4326_2080433960,H,0.00,"(0.0, 0.0)","(0.0, 0.0)",0.00,0.00
flood_DERP_RW_H_4326_2080434150,H,0.00,"(0.0, 0.0)","(0.0, 0.0)",0.00,0.00
flood_DERP_RW_H_4326_2080434280,H,0.00,"(0.006875978474421102, 0.011583223409890188)","(0.006875978474421102, 0.011583223409890188)",0.01,0.01
...,...,...,...,...,...,...
flood_DERP_RW_L_4326_2080433960,L,0.00,"(0.0, 0.0)","(0.0, 0.0)",0.00,0.00
flood_DERP_RW_L_4326_2080433740,L,0.00,"(0.0, 0.0)","(0.0, 0.0)",0.00,0.00
flood_DERP_RW_L_4326_2080433520,L,0.00,"(0.0, 0.0)","(0.0, 0.0)",0.00,0.00
flood_DERP_RW_L_4326_2080448120,L,0.00,"(0.6493972790631091, 1.3360491122195552)","(0.6493972790631091, 1.3360491122195552)",0.09,0.09
